# Example Sample Hotel and Flight Booker Agent 

This solution will help you book flight tickets and hotel.  The scenario is a trip London Heathrow LHR Feb 20th 2024 to New York JFK returning Feb 27th 2025 flying economy with British Airways only. I want a stay in a Hilton hotel in New York please provide costs for the flight and hotel'

# Initialize the Azure AI Agent Service and get configuration information from **.env**

### **.env** 

Create a .env file 

**.env** contains the connection string of Azure AI Agent Service, the model used by AOAI, and the corresponding Google API Search service API, ENDPOINT, etc.

- **AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME** = "Your Azure AI Agent Service Model Deployment Name"

[**NOTE**] You will need a model with 100,000 Rate Limit (Tokens per minute)  Rate Limit of 600 (Request per minute)

  You can get model in Azure AI Foundry - Model and Endpoint. 


- **AZURE_AI_AGENT_PROJECT_CONNECTION_STRING** = "Your Azure AI Agent Service Project Connection String"

  You can get the project connection string in your project overview in  AI ​​Foundry Portal Screen.

- **SERPAPI_SEARCH_API_KEY** = "Your SERPAPI Search API KEY"
- **SERPAPI_SEARCH_ENDPOINT** = "Your SERPAPI Search Endpoint"

To get the Model Deployment Name and Project Connection String of Azure AI Agent Service, you need to create Azure AI Agent Service. It is recommended to use [this template](https://portal.azure.com/#create/Microsoft.Template/uri/https%3A%2F%2Fraw.githubusercontent.com%2Ffosteramanda%2Fazure-agent-quickstart-templates%2Frefs%2Fheads%2Fmaster%2Fquickstarts%2Fmicrosoft.azure-ai-agent-service%2Fstandard-agent%2Fazuredeploy.json) to create it directly （***Note:*** Azure AI Agent Service is currently set in a limited region. It is recommended that you refer to [this link](https://learn.microsoft.com/en-us/azure/ai-services/agents/concepts/model-region-support) to set the region)

Agent needs to access SERPAPI. It is recommended to register using [this link](https://serpapi.com/searches). After registration, you can obtain a unique API KEY and ENDPOINT

# Setup 

To run this notebook, you will need to install the following libraries. Here is a list of the required libraries and the corresponding pip install commands:

azure-identity: For Azure authentication.
requests: For making HTTP requests.
semantic-kernel: For the semantic kernel framework (assuming this is a custom or specific library, you might need to install it from a specific source or repository).

In [ ]:
!pip install azure-identity
!pip install requests
!pip install semantic-kernel
!pip install --upgrade semantic_kernel
!pip install azure-cli


# Explanation: 
import asyncio: This imports the asyncio module, which provides support for asynchronous programming in Python. It allows you to write concurrent code using the async and await syntax.
from typing import Annotated: This imports the Annotated type from the typing module. Annotated is used to add metadata to type hints, which can be useful for various purposes such as validation, documentation, or tooling.

In [ ]:
import asyncio
from typing import Annotated

# Explanation:

Load your .env file setting and resources please ensure you have added your keys and setting and created a local .env file.

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Login to Azure 

You Now need to login into Azure Open a terminal and run the `az login` command

# Explanation:

Import Statement: from azure.identity.aio import DefaultAzureCredential: This imports the DefaultAzureCredential class from the azure.identity.aio module. The aio part of the module name indicates that it is designed for asynchronous operations.

Purpose of DefaultAzureCredential: The DefaultAzureCredential class is part of the Azure SDK for Python. It provides a default way to authenticate with Azure services. It attempts to authenticate using multiple methods in a specific order, such as environment variables, managed identity, and Azure CLI credentials.

Asynchronous Operations:The aio module indicates that the DefaultAzureCredential class supports asynchronous operations. This means you can use it with asyncio to perform non-blocking authentication requests.

In [ ]:
from azure.identity.aio import DefaultAzureCredential

# Explanation:
from semantic_kernel.agents.azure_ai import AzureAIAgent, AzureAIAgentSettings: This imports the AzureAIAgent and AzureAIAgentSettings classes from the semantic_kernel.agents.azure_ai module. These classes are likely used to configure and interact with an AI agent provided by Azure.

from semantic_kernel.contents.chat_message_content import ChatMessageContent: This imports the ChatMessageContent class from the semantic_kernel.contents.chat_message_content module. This class is likely used to handle the content of chat messages.

from semantic_kernel.contents.utils.author_role import AuthorRole: This imports the AuthorRole class from the semantic_kernel.contents.utils.author_role module. This class is likely used to define the role of the author in a chat message (e.g., user, assistant).

from semantic_kernel.functions.kernel_function_decorator import kernel_function: This imports the kernel_function decorator from the semantic_kernel.functions.kernel_function_decorator module. This decorator is likely used to mark functions as kernel functions, which can be used within the semantic kernel framework.

# Purpose of the Imports:
These imports set up the necessary components for working with an AI agent and handling chat messages within the semantic kernel framework. Here's a brief overview of each component:

AzureAIAgent: This class is likely used to create and manage an AI agent that interacts with Azure AI services.

AzureAIAgentSettings: This class is likely used to configure settings for the AzureAIAgent, such as authentication credentials and service endpoints.

ChatMessageContent: This class is likely used to represent the content of chat messages, including text, metadata, and other relevant information.

AuthorRole: This class is likely used to define the role of the author in a chat message, such as whether the message was sent by the user or the AI assistant.

kernel_function: This decorator is likely used to mark functions as kernel functions, which can be used within the semantic kernel framework. Kernel functions are typically used to perform specific tasks or operations within the framework.

In [ ]:
from semantic_kernel.agents.azure_ai import AzureAIAgent, AzureAIAgentSettings
from semantic_kernel.contents.chat_message_content import ChatMessageContent
from semantic_kernel.contents.utils.author_role import AuthorRole
from semantic_kernel.functions.kernel_function_decorator import kernel_function

# Explanation:

This is a class imported from the semantic_kernel.agents.azure_ai module. It is used to configure settings for an AI agent that interacts with Azure AI services.

create() Method: The create() method is a class method of AzureAIAgentSettings. It is used to create and initialize an instance of AzureAIAgentSettings with default or predefined settings.

ai_agent_settings Variable: The variable ai_agent_settings is assigned the instance of AzureAIAgentSettings created by the create() method. This instance contains the configuration settings needed to set up and use an Azure AI agent.

# Purpose:
The purpose of this line is to create a configuration object (ai_agent_settings) that holds the necessary settings for an Azure AI agent. These settings might include authentication credentials, service endpoints, and other configuration parameters required to interact with Azure AI services.

In [ ]:
ai_agent_settings = AzureAIAgentSettings.create()


# Explanation:
import requests: This statement imports the requests library, which is a popular third-party library in Python used for making HTTP requests.

Purpose: The requests library simplifies the process of sending HTTP requests and handling responses. It provides a user-friendly interface for interacting with web services and APIs.

Common Use Cases: Sending HTTP Requests: You can use the requests library to send various types of HTTP requests, such as GET, POST, PUT, DELETE, etc.

Handling Responses:The library makes it easy to handle responses, including reading response content, headers, status codes, and more.
Working with APIs:

The requests library is commonly used to interact with RESTful APIs, allowing you to send data to and retrieve data from web services.

In [ ]:
import requests

# Explanation:
This is a variable that stores the API key for accessing a SERP (Search Engine Results Page) API service. An API key is a unique identifier used to authenticate requests associated with your account.

Purpose: The purpose of this line is to store the API key in a variable so that it can be used to authenticate requests to the SERP API service. The API key is required to access the service and perform searches.
How to Get a SERP API Key: To get a SERP API key, follow these general steps at https://serpapi.com (the exact steps may vary depending on the specific SERP API service you are using):

Choose a SERP API Service: There are several SERP API services available, such as SerpAPI, Google Custom Search JSON API, and others. Choose the one that best fits your needs.

Sign Up for an Account: Go to the website of the chosen SERP API service and sign up for an account. You may need to provide some basic information and verify your email address.

Create an API Key: After signing up, log in to your account and navigate to the API section or dashboard. Look for an option to create or generate a new API key.
Copy the API Key to your .env file.

In [ ]:
SERP_API_KEY='SERPAPI_SEARCH_API_KEY'

# Explanation:
BASE_URL: This is a variable that stores the base URL for the SERP API endpoint. The variable name BASE_URL is a convention used to indicate that this URL is the starting point for making API requests.
'https://serpapi.com/search':

This is the actual URL string assigned to the BASE_URL variable. It represents the endpoint for performing search queries using the SERP API.

# Purpose:
The purpose of this line is to define a constant that holds the base URL for the SERP API. This URL will be used as the starting point for constructing API requests to perform search operations.

# Usage:
By defining the base URL in a variable, you can easily reuse it throughout your code whenever you need to make requests to the SERP API. This makes your code more maintainable and reduces the risk of errors from hardcoding the URL in multiple places. The current example is https://serpapi.com/search?engine=bing which is using Bing search API. Different API can be selected at https://Serpapi.com

In [ ]:
BASE_URL = 'https://serpapi.com/search?engine=bing'

# Explanation:

This is where you agent code is located.

Class Definition: class BookingPlugin:: Defines a class named BookingPlugin that contains methods for booking hotels and flights.
Docstring: """Booking Plugin for customers""": A docstring that describes the purpose of the BookingPlugin class.
Hotel Booking Method:

@kernel_function(description="booking hotel"): A decorator that describes the function as a kernel function for booking hotels.
def booking_hotel(self, query: Annotated[str, "The name of the city"], check_in_date: Annotated[str, "Hotel Check-in Time"], check_out_date: Annotated[str, "Hotel Check-out Time"]) -> Annotated[str, "Return the result of booking hotel information"]:: Defines a method for booking hotels with annotated parameters and return type.

The method constructs a dictionary of parameters for the hotel booking request and sends a GET request to the SERP API. It checks the response status and returns the hotel properties if successful, or None if the request failed.

Flight Booking Method: @kernel_function(description="booking flight"): A decorator that describes the function as a kernel function for booking flights.
def booking_flight(self, origin: Annotated[str, "The name of Departure"], destination: Annotated[str, "The name of Destination"], outbound_date: Annotated[str, "The date of outbound"], return_date: Annotated[str, "The date of Return_date"]) -> Annotated[str, "Return the result of booking flight information"]:: Defines a method for booking flights with annotated parameters and return type.

The method constructs dictionaries of parameters for the outbound and return flight requests and sends GET requests to the SERP API. It checks the response status and appends the flight information to the result string if successful, or prints an error message if the request failed. The method returns the result string containing the flight information.


In [ ]:
# Define Booking Plugin
class BookingPlugin:
    """Booking Plugin for customers"""

    @kernel_function(description="booking hotel")
    def booking_hotel(self, query: Annotated[str, "The name of the city"], check_in_date: Annotated[str, "Hotel Check-in Time"], check_out_date: Annotated[str, "Hotel Check-out Time"]) -> Annotated[str, "Return the result of booking hotel information"]:
        """
        Function to book a hotel.
        Parameters:
        - query: The name of the city
        - check_in_date: Hotel Check-in Time
        - check_out_date: Hotel Check-out Time
        Returns:
        - The result of booking hotel information
        """

        # Define the parameters for the hotel booking request
        params = {
            "engine": "google_hotels",
            "q": query,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adults": "1",
            "currency": "GBP",
            "gl": "uk",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request to the SERP API
        response = requests.get(BASE_URL, params=params)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the response content as JSON
            response = response.json()
            # Return the properties from the response
            return response["properties"]
        else:
            # Return None if the request failed
            return None

    @kernel_function(description="booking flight")
    def booking_flight(self, origin: Annotated[str, "The name of Departure"], destination: Annotated[str, "The name of Destination"], outbound_date: Annotated[str, "The date of outbound"], return_date: Annotated[str, "The date of Return_date"]) -> Annotated[str, "Return the result of booking flight information"]:
        """
        Function to book a flight.
        Parameters:
        - origin: The name of Departure
        - destination: The name of Destination
        - outbound_date: The date of outbound
        - return_date: The date of Return_date
        - airline: The preferred airline carrier
        - hotel_brand: The preferred hotel brand
        Returns:
        - The result of booking flight information
        """
        
        # Define the parameters for the outbound flight request
        go_params = {
            "engine": "google_flights",
            "departure_id": "destination",
            "arrival_id": "origin",
            "outbound_date": "outbound_date",
            "return_date": "return_date",
            "currency": "GBP",
            "hl": "en",
            "airline": "airline",
            "hotel_brand": "hotel_brand",
            "api_key": "SERP_API_KEY"
        }
 
 
 
        print(go_params)

        # Send the GET request for the outbound flight
        go_response = requests.get(BASE_URL, params=go_params)

        # Initialize the result string
        result = ''

        # Check if the outbound flight request was successful
        if go_response.status_code == 200:
            # Parse the response content as JSON
            response = go_response.json()
            # Append the outbound flight information to the result
            result += "# outbound \n " + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Define the parameters for the return flight request
        back_params = {
            #"engine": "google_flights",
            "departure_id": destination,
            "arrival_id": origin,
            "outbound_date": outbound_date,
            "return_date": return_date,
            "currency": "GBP",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request for the return flight
        back_response = requests.get(BASE_URL, params=back_params)

        # Check if the return flight request was successful
        if back_response.status_code == 200:
            # Parse the response content as JSON
            response = back_response.json()
            # Append the return flight information to the result
            result += "\n # return \n" + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Print the result
        print(result)

        # Return the result
        return result


# Explanation:
Import Statements: Import necessary modules for Azure credentials, AI agent, chat message content, author role, and kernel function decorator.

Asynchronous Context Manager: async with (DefaultAzureCredential() as creds, AzureAIAgent.create_client(credential=creds, conn_str="...") as client,):: This sets up an asynchronous context manager to handle Azure credentials and create an AI agent client.

Agent Name and Instructions: AGENT_NAME = "BookingAgent": Defines the name of the agent.
AGENT_INSTRUCTIONS = """...""": Provides detailed instructions for the agent on how to handle booking requests.

Create Agent Definition: agent_definition = await client.agents.create_agent(...): Creates an agent definition with the specified model, name, and instructions.

Create AzureAI Agent: agent = AzureAIAgent(...): Creates an AzureAI agent using the client and agent definition.

Add Booking Plugin: agent.kernel.add_plugin(BookingPlugin(), plugin_name="booking"): Adds the BookingPlugin to the agent's kernel.

Create Thread: thread = await client.agents.create_thread(): Creates a new thread for the agent.

User Inputs: user_inputs = ["..."]: Defines a list of user inputs for the agent to process.

Process User Inputs:For each user input, add it as a chat message, invoke the agent, and print the agent's response.
Clean Up:

In the finally block, delete the thread and agent to clean up resources.

In [ ]:
# Import necessary modules
from azure.identity.aio import DefaultAzureCredential
from semantic_kernel.agents.azure_ai import AzureAIAgent, AzureAIAgentSettings
from semantic_kernel.contents.chat_message_content import ChatMessageContent
from semantic_kernel.contents.utils.author_role import AuthorRole
from semantic_kernel.functions.kernel_function_decorator import kernel_function

# Azure AI Setting
async with (
     DefaultAzureCredential() as creds,
            AzureAIAgent.create_client(
                credential=creds,
                conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
            ) as client,
):    
    
    # Define the agent's name and instructions
    AGENT_NAME = "BookingAgent"
    AGENT_INSTRUCTIONS = """
    You are a booking agent, help me to book flights or hotels.

    Thought: Understand the user's intention and confirm whether to use the reservation system to complete the task.

    Action:
    - If booking a flight, convert the departure name and destination name into airport codes.
    - If booking a hotel or flight, use the corresponding API to call. Ensure that the necessary parameters are available. If any parameters are missing, use default values or assumptions to proceed.
    - If it is not a hotel or flight booking, respond with the final answer only.
    - Output the results using a markdown table:
    - For flight bookings, separate the outbound and return contents and list them in the order of Departure_airport Name | Airline | Flight Number | Departure Time | Arrival_airport Name | Arrival Time | Duration | Airplane | Travel Class | Price (USD) | Legroom | Extensions | Carbon Emissions (kg).
    - For hotel bookings, list them in the order of Properties Name | Properties description | check_in_time | check_out_time | prices | nearby_places | hotel_class | gps_coordinates.
"""

    # Create agent definition with the specified model, name, and instructions
    agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=AGENT_NAME,
        instructions=AGENT_INSTRUCTIONS,
    )

    # Create the AzureAI Agent using the client and agent definition
    agent = AzureAIAgent(
        client=client,
        definition=agent_definition,
        # Optionally configure polling options
        # polling_options=RunPollingOptions(run_polling_interval=timedelta(seconds=1)),
    )

    # Add the BookingPlugin to the agent's kernel
    agent.kernel.add_plugin(BookingPlugin(), plugin_name="booking")

    # Create a new thread for the agent
    thread = await client.agents.create_thread()

    # This is your prompt for the activity or task you want to complete 
    # Define user inputs for the agent to process we have provided some example prompts to test and validate 
    user_inputs = [
        # "Can you tell me the round-trip air ticket from  London to New York JFK aiport, the departure time is February 17, 2025, and the return time is February 23, 2025"
        # "Book a hotel in New York from Feb 20,2025 to Feb 24,2025"
        "Help me book flight tickets and hotel for the following trip London Heathrow LHR Feb 20th 2025 to New York JFK returning Feb 27th 2025 flying economy with British Airways only. I want a stay in a Hilton hotel in New York please provide costs for the flight and hotel"
        # "I have a business trip from London LHR to New York JFK on Feb 20th 2025 to Feb 27th 2025, can you help me to book a hotel and flight tickets"
    ]

    try:
        # Process each user input
        for user_input in user_inputs:
            # Add the user input as a chat message
            await agent.add_chat_message(
                thread_id=thread.id, message=ChatMessageContent(role=AuthorRole.USER, content=user_input)
            )
            print(f"# User: '{user_input}'")
            # Invoke the agent for the specified thread
            async for content in agent.invoke(
                thread_id=thread.id,
                temperature=0.2,  # Override the agent-level temperature setting with a run-time value
            ):
                if content.role != AuthorRole.TOOL:
                    print(f"# Agent: {content.content}")
    finally:
        # Clean up by deleting the thread and agent
        await client.agents.delete_thread(thread.id)
        await client.agents.delete_agent(agent.id)